In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3f65df92f31d331dc373b31c1a915ce21c0fbae612c0fde7279f4be9d87deb00
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-28 03:46:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.59MB/s    in 0.6s    

2023-01-28 03:46:10 (1.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Create Spark Session "M17-Amazon-Challenge"
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...|2015-08-31 00:00:00|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend| 

### Create DataFrames to match tables

In [7]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.functions import count

In [8]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id") \
  .agg(count("*"))
  # .withColumnRenamed("count(customer_id)", "customer_count")

customers_df.show()

+-----------+--------+
|customer_id|count(1)|
+-----------+--------+
|   16747249|       4|
|     108460|       1|
|   11720077|      31|
|     437936|       1|
|     128131|       1|
|   39647402|       1|
|   13730111|       8|
|   43783459|       1|
|   12493167|     111|
|   40325486|      92|
|   44618558|       4|
|   17956020|       1|
|   10093406|       3|
|   40430762|       1|
|     134924|       1|
|   52427934|       1|
|   12879980|      15|
|   27680937|       7|
|   16186289|       1|
|   20326078|       1|
+-----------+--------+
only showing top 20 rows



In [9]:
customers_df = customers_df.select(["customer_id", "count(1)"]).withColumnRenamed("count(1)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16747249|             4|
|     108460|             1|
|   11720077|            31|
|     437936|             1|
|     128131|             1|
|   39647402|             1|
|   13730111|             8|
|   43783459|             1|
|   12493167|           111|
|   40325486|            92|
|   44618558|             4|
|   17956020|             1|
|   10093406|             3|
|   40430762|             1|
|     134924|             1|
|   52427934|             1|
|   12879980|            15|
|   27680937|             7|
|   16186289|             1|
|   20326078|             1|
+-----------+--------------+
only showing top 20 rows



In [10]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|014241543X|           If I Stay|
|0062091158|     King (Prophecy)|
|1250056608|Rough Justice: Si...|
|0340909129|Do It Tomorrow an...|
|1937013200|Savage Worlds Del...|
|019513172X|Nothing but Chris...|
|0736432841|Anna & Elsa #1: A...|
|1940484138|Common Core Asses...|
|0307476073|Wild: From Lost t...|
|1416299025|Outlander 2016 Bo...|
|1743214022|Lonely Planet Cro...|
|B001IF2MG6|Calligraphy the K...|
|B00UZGLL02|Mermaid Juniors' ...|
|0060192674|Everything You Al...|
|1423422198|Swing Favorites -...|
|1572840994|Gluten-Free Bakin...|
|8888166122|Italia in Cucina:...|
|0803740123|             Tracked|
|0062433652|Go Set a Watchman...|
|0590043749|State-by-State Gu...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).withColumn('new_review_date', to_date(col("review_date"), "yyyy-mm-dd"))
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+---------------+
|     review_id|customer_id|product_id|product_parent|        review_date|new_review_date|
+--------------+-----------+----------+--------------+-------------------+---------------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342|2015-08-31 00:00:00|     2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122|2015-08-31 00:00:00|     2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048|2015-08-31 00:00:00|     2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151|2015-08-31 00:00:00|     2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372|2015-08-31 00:00:00|     2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774|2015-08-31 00:00:00|     2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591|2015-08-31 00:00:00|     2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353|2015-08-31 00:00:00|     2015-08-31|

In [12]:
review_id_df = review_id_df.select(["review_id", "customer_id", "product_id", "product_parent", "new_review_date"]).withColumnRenamed("new_review_date", "review_date")
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048| 2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260|0679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950|0983990662|     222024699| 2015-08

In [13]:
# Create the vine_table. DataFrame
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [14]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [18]:
vine_table = vine_table.withColumn('star_rating_int', col("star_rating").cast("int"))

In [19]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- star_rating_int: integer (nullable = true)



In [21]:
vine_table = vine_table.select(["review_id", "star_rating_int", "helpful_votes", "total_votes", "vine", "verified_purchase"]).withColumnRenamed("star_rating_int", "star_rating")
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [22]:
# Configure settings for RDS
from getpass import getpass
password = getpass('type password')
mode = "append"
jdbc_url="jdbc:postgresql://database-ohio.cimrlgf1ywh2.us-east-2.rds.amazonaws.com:5432/module_17_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

type password··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
products_df = products_df.drop_duplicates(['product_id'])

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [23]:
# Write vine_df to table in RDS
# 11 minutes
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)